# Import libraries

In [ ]:
!pip install openai cohere tiktoken typing-extensions==4.5.0

from google.colab import userdata

import inspect
import os
import base64
import numpy as np
import pandas as pd

from openai import OpenAI

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Authentication

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

# Import dataset

In [ ]:
total_defense_memes_test = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_test.csv')

# Define function to add definitions

In [ ]:
def definition(x):
    if x == 'Military Defence':
        return """Singapore's Military Defence: Strong and formidable defence force made up of Regulars and National Servicemen, and supported by the entire Singapore."""
    elif x == 'Civil Defence':
        return """Singapore's Civil Defence: Collective effort of the Singaporean society to spot signs of threats, respond effectively and recover quickly from crisis."""
    elif x == 'Economic Defence':
        return """Singapore's Economic Defence: Strong and resilient Singaporean economy that is globally competitive and able to bounce back from any crisis."""
    elif x == 'Social Defence':
        return """Singapore's Social Defence: Bonds that unite Singaporeans, built on trust and understanding among people of different races and religions, living in harmony and looking out for one another."""
    elif x == 'Psychological Defence':
        return """Singapore's Psychological Defence: The will and resolve to defend the Singaporean way of life and interests, the fighting spirit to overcome challenges together."""
    elif x == 'Digital Defence':
        return """Singapore's Digital Defence: Being secure, alert and responsible online."""

In [ ]:
total_defense_memes_test['pillar_w_definition'] = total_defense_memes_test['pillar'].apply(lambda x: definition(x))

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x == 'bad':
        return 'Against'
    elif x == 'neutral':
        return 'Neutral'
    elif  x == 'good':
        return 'Supportive'
    else:
        return None

# Set prompts

In [ ]:
def prompt_vanilla(pillar, pillar_w_definition):
    return inspect.cleandoc(f"""
    {pillar_w_definition}
    Indicate whether this meme is bad, neutral or good towards Singapore's {pillar}.
    Constraint: Without using any other words, answer either bad, neutral, good.""")

# Define a function to generate text using the `gpt-4-vision-preview` model

In [ ]:
# Function to encode the image. via
# https://platform.openai.com/docs/guides/vision
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Prevent rate limit errors. via
# https://github.com/openai/openai-cookbook/blob/main/examples/
# How_to_handle_rate_limits.ipynb
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
# Call the OPENAI API. via
# https://platform.openai.com/docs/guides/vision
def prediction(image, pillar, pillar_w_definition):
    completion = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt_vanilla(pillar, pillar_w_definition)},
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encode_image(image)}",
            },
            },
        ],
        }
    ],
    max_tokens=4096,
    temperature=0,
    seed=1,
    )

    return completion.choices[0].message.content, completion.system_fingerprint, completion.usage.prompt_tokens, completion.usage.completion_tokens

# Call the `prediction` function and save inferences

In [ ]:
%%time
df_temp = total_defense_memes_test.copy(deep=True)
# Create new columns by applying function. via
# https://stackoverflow.com/a/52363890
df_temp[['prediction', 'fingerprint_prediction', 'prompt_tokens_prediction', 'completion_tokens_prediction']] = df_temp.apply(lambda row: prediction(row.image, row.pillar, row.pillar_w_definition), axis='columns', result_type='expand')
df_temp.to_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_test_zero_shot_inference.csv', index=False)

CPU times: user 26.6 s, sys: 3.06 s, total: 29.7 s
Wall time: 52min 57s


In [ ]:
df_temp['prediction'] = df_temp['prediction'].apply(lambda x: remap(x))
print(df_temp['prediction'].value_counts())
print(df_temp['prediction'].isna().sum())
df_temp['prediction'] = df_temp['prediction'].apply(lambda x: x if x is not None else np.random.choice(['Against', 'Neutral', 'Supportive']))
print(f1_score(df_temp['stance'].values, df_temp['prediction'].values, labels=['Against', 'Neutral', 'Supportive'], average='macro'))
print(classification_report(df_temp['stance'].values, df_temp['prediction'].values, labels=['Against', 'Neutral', 'Supportive']))

Neutral       468
Against       247
Supportive     55
Name: prediction, dtype: int64
1
0.6570743002128644
              precision    recall  f1-score   support

     Against       0.82      0.58      0.68       352
     Neutral       0.62      0.87      0.72       332
  Supportive       0.73      0.47      0.57        87

    accuracy                           0.69       771
   macro avg       0.72      0.64      0.66       771
weighted avg       0.72      0.69      0.68       771

